In [ ]:
import json
import requests
from PIL import Image
from tensorflow.keras.utils import img_to_array
import numpy as np
import tensorflow as tf

def prepare_image(src_img, target):
	if src_img.mode != "RGB":
		src_img = src_img.convert("RGB")
	src_img = src_img.resize(target)

	src_img = img_to_array(src_img)
	src_img = np.expand_dims(src_img, axis=0)

	return src_img

pb_path = "./model_saved/tf/eyes/efficientnet_10/model-003-0.481325-0.366379.h5"
model = tf.keras.models.load_model(pb_path)

In [ ]:
img_path = "crop_D0_0c8b3399-60a5-11ec-8402-0a7404972c70.png"
src_image = prepare_image(Image.open(img_path), target=(224, 224))
model.predict(src_image.tolist())

In [ ]:
import io
import base64
import requests
from PIL import Image

# FastAPI URL
url = "http://localhost:5000/image"
# 요청
result = requests.get(url)
# 딕셔너리 변환
res = json.loads(result.content)
# 이미지 Bytes 리스트
bytes_list = list(map(lambda x: base64.b64decode(x), res))
# 이미지 Pillow Image 리스트
image_list = list(map(lambda x: Image.open(io.BytesIO(x)), bytes_list))
image_list[0]

In [ ]:
from fastapi import APIRouter
from fastapi.responses import JSONResponse
from PIL import Image
import requests
import base64
import io

router = APIRouter()

def from_image_to_bytes(img):
    """
    pillow image 객체를 bytes로 변환
    """
    # Pillow 이미지 객체를 Bytes로 변환
    imgByteArr = io.BytesIO()
    img.save(imgByteArr, format=img.format)
    imgByteArr = imgByteArr.getvalue()
    # Base64로 Bytes를 인코딩
    encoded = base64.b64encode(imgByteArr)
    # Base64로 ascii로 디코딩
    decoded = encoded.decode('ascii')
    return decoded

@router.get("/image/")
async def func():
    # 이미지 주소
    img_url = "https://dimg.donga.com/ugc/CDB/WEEKLY/Article/5b/b3/22/85/5bb32285000ed2738de6.jpg"
    # 이미지 불러오기
    img = Image.open(requests.get(img_url, stream=True).raw)
    # 이미지 데이터 타입 변환
    img_converted = from_image_to_bytes(img)
    # 이미지 리스트 생성
    img_list = [img_converted, img_converted]
    return JSONResponse(img_list)